## Partial Dependence (PDP) and Individual Conditional Expectation (ICE) plots

In this example, we train a classification model with the Adult Census Income dataset. Then we treat the model as a blackbox model and calculate the PDP and ICE plots for some selected categorical and numeric features. 

This dataset can be used to predict whether annual income exceeds $50,000/year or not based on demographic data from the 1994 U.S. Census. The dataset we're reading contains 32,561 rows and 14 columns/features.

[More info on the dataset here](https://archive.ics.uci.edu/ml/datasets/Adult)

We will train a classification model with a target - income >= 50K.

---

**Partial Dependence Plot (PDP) ** function at a particular feature value represents the average prediction if we force all data points to assume that feature value.

**Individual Conditional Expectation (ICE)** plots display one line per instance that shows how the instance’s prediction changes when a feature changes. One line represents the predictions for one instance if we vary the feature of interest.

PDP and ICE plots visualize and help to analyze the interaction between the target response and a set of input features of interest. It is essential when you are building a Machine Learning model to understand model behavior and how certain features influences overall prediction. One of the most popular use-cases is analyzing feature importance.

---
Python dependencies:

matplotlib==3.2.2

numpy==1.19.2

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
import pyspark.sql.functions as F
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from synapse.ml.explainers import ICETransformer

import matplotlib.pyplot as plt

### Read and prepare the dataset

In [ ]:
df = spark.read.parquet("wasbs://publicwasb@mmlspark.blob.core.windows.net/AdultCensusIncome.parquet")
df.show(20)

In [ ]:
categorical_features = ["race", "workclass", "marital-status", "education", "occupation", "relationship", "native-country", "sex"]
numeric_features = ["age", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
string_indexer_outputs = [feature + "_idx" for feature in categorical_features]
one_hot_encoder_outputs = [feature + "_enc" for feature in categorical_features]

pipeline = Pipeline(stages=[
    StringIndexer().setInputCol("income").setOutputCol("label").setStringOrderType("alphabetAsc"),
    StringIndexer().setInputCols(categorical_features).setOutputCols(string_indexer_outputs),
    OneHotEncoder().setInputCols(string_indexer_outputs).setOutputCols(one_hot_encoder_outputs),
    VectorAssembler(inputCols=one_hot_encoder_outputs+numeric_features, outputCol="features"),
    GBTClassifier(weightCol="fnlwgt")])

In [ ]:
display(df.groupBy("education-num").count())

### Fit the model and view the predictions

In [ ]:
model = pipeline.fit(df)

Check that model makes sense and has reasonable output. For this, we will check the model performance by calculating the ROC-AUC score.

In [ ]:
data = model.transform(df)
data.select('income', 'probability', 'prediction').show(20)

In [ ]:
eval_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction")
eval_auc.evaluate(data)

## PDP

\\(X_S\\) - set of input features of interest, \\(X_C\\) - its complement.

The partial dependence of the response \\(f\\) at a point \\(x_S\\) is defined as:

$$ pd_{X_S}(x_S) = \mathsf{E} X_C [f(x_S, X_C)] = \int f(x_S, x_C) p(x_C)dx_C$$

where \\(f(x_S, x_C)\\)  is the response function for a given sample whose values are defined by \\(x_S\\) for the features in \\(X_S\\) (i.e. the features you want to explain), and by \\(x_C\\) for the features in \\(X_C\\) (i.e. features that are not being analyzed).

The compuation method estimates the above integaral by computing an average over the dataset \\(X\\):

$$pd_{X_S}(x_S) \approx \frac{1}{n_{samples}} \sum_{i=1}^n f(x_S, x_C^{(i)}) $$

where \\(x_C^{(i)}\\) is the value of the i-th sample for the features in \\(X_C\\). For each value of \\(x_S\\), this method requires a full pass over the dataset \\(X\\).

---

We will show how features "sex", "education", "worklass", "occupation" (categorical feautures) and "education-num" and "age" (numeric features) affect the prediction of the income exceeds $50,000/year.

--- 

Source: https://christophm.github.io/interpretable-ml-book/pdp.html

### Setup the transformer for PDP

In [ ]:
pdp = ICETransformer(model=model, targetCol="probability", kind="average", targetClasses=[1],
                     categoricalFeatures=categorical_features, numericFeatures=numeric_features, numSamples=50)

PDP is a spark transformer, the function **transform** returns the schema of (1 row * number features to explain) which contains dependence for the given feature in a format: feature_value -> dependence (in our case probability).

In [ ]:
output_pdp = pdp.transform(df)
output_pdp.show()

### Visualization

In [ ]:
# Helper functions for visualization

def get_pandas_df_from_column(df, col_name):
  keys_df = df.select(F.explode(F.map_keys(F.col(col_name)))).distinct()
  keys = list(map(lambda row: row[0], keys_df.collect()))
  key_cols = list(map(lambda f: F.col(col_name).getItem(f).alias(str(f)), keys))
  final_cols = key_cols
  pandas_df = df.select(final_cols).toPandas()
  return pandas_df

def plot_dependence_for_categorical(df, col, col_int=True, figsize=(20, 5)):
  dict_values = {}
  col_names = list(df.columns)

  for col_name in col_names:
    dict_values[col_name] = df[col_name][0].toArray()[0]
    marklist= sorted(dict_values.items(), key=lambda x: int(x[0]) if col_int else x[0]) 
    sortdict=dict(marklist)

  fig = plt.figure(figsize = figsize)
  plt.bar(sortdict.keys(), sortdict.values())

  plt.xlabel(col, size=13)
  plt.ylabel("Dependence")
  plt.show()
  
def plot_dependence_for_numeric(df, col, col_int=True, figsize=(20, 5)):
  dict_values = {}
  col_names = list(df.columns)

  for col_name in col_names:
    dict_values[col_name] = df[col_name][0].toArray()[0]
    marklist= sorted(dict_values.items(), key=lambda x: int(x[0]) if col_int else x[0]) 
    sortdict=dict(marklist)

  fig = plt.figure(figsize = figsize)

  
  plt.plot(list(sortdict.keys()), list(sortdict.values()))

  plt.xlabel(col, size=13)
  plt.ylabel("Dependence")
  plt.ylim(0.0)
  plt.show()
  

#### Example 1: "Age"

We can observe non-linear dependency. Income rapidly grows from 24-38 age, after 58 it slightly drops and from 68 remains stable.

In [ ]:
output_pdp.show()

In [ ]:
df_education_num = get_pandas_df_from_column(output_pdp, 'age_dependence')
plot_dependence_for_numeric(df_education_num, 'age')

#### Example 2: "marital-status"

According to the result, the model treats "married-cv-spouse" as one category and all others as a second category. It looks reasonable, taking into account that GBT has a tree structure.

If the model picks "divorced" as one category and the rest features as the second category- then most likely there is an error and some bias in data.

In [ ]:
df_occupation = get_pandas_df_from_column(output_pdp, 'workclass_dependence')
plot_dependence_for_categorical(df_occupation, 'marital-status', False, figsize=(30, 5))

#### Example 3: "capital-gain"

Firstly we run PDP with default parameters for rangeMin and rangeMax. We can see that this representation is not useful, it is not granulated enough, because it was dynamically computed from the data. That is why we set rangeMin = 0 and rangeMax = 10000 to visualize more granulated interpretations for the part we're interested in.

On the second graph we can observe how capital-gain affects the dependence.

In [ ]:
df_education_num = get_pandas_df_from_column(output_pdp, 'capital-gain_dependence')
plot_dependence_for_numeric(df_education_num, 'capital-gain_dependence')

In [ ]:
pdp_cap_gain = ICETransformer(model=model, targetCol="probability", kind="average", targetClasses=[1], 
                              numericFeatures=[{"name": "capital-gain", "numSplits": 20, "rangeMin": 0.0,
                                                 "rangeMax": 10000.0}], numSamples=50)

output_pdp_cap_gain = pdp_cap_gain.transform(df)

In [ ]:
df_education_num_gain = get_pandas_df_from_column(output_pdp_cap_gain, 'capital-gain_dependence')
plot_dependence_for_numeric(df_education_num_gain, 'capital-gain_dependence')

### Conclusions

**Advantages:**

1) Plots is intuitive.

2) PDPs perfectly represent how the feature influences the prediction on average (for not correlated features).

3) Plots are easy to implement.

**Disadvantages:**

1) The realistic maximum number of features in a partial dependence function is two.

2) Some PD plots do not show the feature distribution.

3) The assumption of independence is the biggest issue with PD plots.

4) PD plots only show the average marginal effects.

## ICE

\\(X_S\\) - set of input features of interest, \\(X_C\\) - its complement.


The equivalent to a PDP for individual data instances is called individual conditional expectation (ICE) plot. A PDP is the average of the lines of an ICE plot.

The values for a line (and one instance) can be computed by keeping all other features the same, creating variants of this instance by replacing the feature’s value with values from a grid and making predictions with the black box model for these newly created instances. 

For each instance in $$ \{ (x_{S}^{(i)},x_{C}^{(i)}) \}_{i=1}^N$$ the curve \\(\hat{f}_S^{(i)}\\) is plotted against \\(x_S^{(i)} \\), while \\( x_C^{(i)}\\) remains fixed.

---


We will show the same features as for PDP to show a difference: "sex", "education", "worklass", "occupation" (categorical feautures) and "education-num" and "age" (numeric features)

---
Source: https://christophm.github.io/interpretable-ml-book/ice.html

### Setup the transformer for ICE

In [ ]:
ice = ICETransformer(model=model, targetCol="probability", targetClasses=[1], 
                     categoricalFeatures=categorical_features, numericFeatures=numeric_features, numSamples=50)

In [ ]:
output = ice.transform(df)
output.show()

### Visualization

In [ ]:
# Helper functions for visualization
from math import pi

from collections import defaultdict

def plot_ice_numeric(df, col, col_int=True, figsize=(20, 10)):
  dict_values = defaultdict(list)
  col_names = list(df.columns)
  num_instances = df.shape[0]
  
  instances_y = {}
  i = 0

  for col_name in col_names:
    for i in range(num_instances):
      dict_values[i].append(df[col_name][i].toArray()[0])
  
  fig = plt.figure(figsize = figsize)
  for i in range(num_instances):
    plt.plot(col_names, dict_values[i], "k")
  
  
  plt.xlabel(col, size=13)
  plt.ylabel("Dependence")
  plt.ylim(0.0)
  
  
  
def plot_ice_categorical(df, col, col_int=True, figsize=(20, 10)):
  dict_values = defaultdict(list)
  col_names = list(df.columns)
  num_instances = df.shape[0]
  
  angles = [n / float(df.shape[1]) * 2 * pi for n in range(df.shape[1])]
  angles += angles [:1]
  
  instances_y = {}
  i = 0

  for col_name in col_names:
    for i in range(num_instances):
      dict_values[i].append(df[col_name][i].toArray()[0])
  
  fig = plt.figure(figsize = figsize)
  ax = plt.subplot(111, polar=True)
  plt.xticks(angles[:-1], col_names)
  
  for i in range(num_instances):
    values = dict_values[i]
    values += values[:1]
    ax.plot(angles, values, "k")
    ax.fill(angles, values, 'teal', alpha=0.1)

  plt.xlabel(col, size=13)
  plt.show()
    

#### Example 1: Numeric feature: "Age"

All curves seem to follow the same course, so there are no obvious interactions. That means that the PDP is already a good summary of the relationships between the displayed features and the predicted income >=50K

In [ ]:
age_dep = get_pandas_df_from_column(output, 'age_dependence')

plot_ice_numeric(age_dep, col_name, figsize=(30, 10))

Helper function

In [ ]:
def overlay_ice_with_pdp(df_ice, df_pdp, col, col_int=True, figsize=(20, 5)):
  dict_values = defaultdict(list)
  col_names_ice = list(df_ice.columns)
  num_instances = df_ice.shape[0]
  
  instances_y = {}
  i = 0

  for col_name in col_names_ice:
    for i in range(num_instances):
      dict_values[i].append(df_ice[col_name][i].toArray()[0])
  
  fig = plt.figure(figsize = figsize)
  for i in range(num_instances):
    plt.plot(col_names_ice, dict_values[i], "k")
    
  dict_values_pdp = {}
  col_names = list(df_pdp.columns)

  for col_name in col_names:
    dict_values_pdp[col_name] = df_pdp[col_name][0].toArray()[0]
    marklist= sorted(dict_values_pdp.items(), key=lambda x: int(x[0]) if col_int else x[0]) 
    sortdict=dict(marklist)
  
  plt.plot(col_names_ice, list(sortdict.values()), "r", linewidth=5)
  
  
  
  plt.xlabel(col, size=13)
  plt.ylabel("Dependence")
  plt.ylim(0.0)
  plt.show()
  

This shows how PDP visualizes the average dependence. Red line - PDP plot, black lines - ICE plots

In [ ]:
age_df_ice = get_pandas_df_from_column(output, 'age_dependence')
age_df_pdp = get_pandas_df_from_column(output_pdp, 'age_dependence')

overlay_ice_with_pdp(age_df_ice, age_df_pdp, col='age_dependence', figsize=(30, 10))

#### Example 2: Categorical feature: "occupation"

In [ ]:
occupation_dep = get_pandas_df_from_column(output, 'occupation_dependence')


plot_ice_categorical(occupation_dep, col_name, figsize=(30, 10))

### Conclusions


**Advantages:**

1) Plots are intuitive to understand. One line represents the predictions for one instance if we vary the feature of interest.

2) ICE curves can uncover more complex relationships.

**Disadvantages:**

1) ICE curves can only display one feature meaningfully - otherwise you should overlay multiple surfaces.

2) Some points in the lines might be invalid data points according to the joint feature distribution. It causes by correlations between features.

3) In ICE plots it might not be easy to see the average.

## Summary

Partial dependence plots (PDP) and Individual Conditional Expectation (ICE) plots can be used to visualize and analyze interaction between the target response and a set of input features of interest.

Both PDPs and ICEs assume that the input features of interest are independent from the complement features, and this assumption is often violated in practice.

ICE shows the dependence on average, but if you want to observe features individually - you can use ICE.

Using examples above we showed how it can be usefull to draw such plots to analyze how machine learning model made their predictions, what was important and how we can interpret the results.